In [3]:
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score 
import fastapi
from typing import Optional

In [2]:
#!pip install fastapi

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.3 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [4]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"] 
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.column_stack((_x, _y)), _result)


DummyClassifier(constant='True', strategy='constant')

In [5]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred, pos_label='True')
    testprecision=precision_score(y_test, y_pred, pos_label='True')
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [6]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [8]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    global model
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    # X = np.rec.fromarrays([_x, _y])[(-1*last_n):]
    X = np.column_stack((_x, _y))[(-1*last_n):]
    Y = np.array(_result)[(-1*last_n):]

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train) 
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model_str:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    global model
    print(model_str, last_n, strategy, k)
    if model_str == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model_str == "KNN" :
        model = KNeighborsClassifier(n_neighbors=k)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

@app.get("/evaluate_model/")
def evaluate_model(last_n:int):
   return {"message": "Item created", "last_n": last_n}

In [9]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [76831]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1 -0.3368229004962464 -0.15646851757795588
True
INFO:     127.0.0.1:59081 - "GET /predict/?id=1&x=-0.3368229004962464&y=-0.15646851757795588 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:59082 - "GET /result/?id=1&result=True HTTP/1.1" 200 OK
2 -0.9015375326817245 0.11718047485328942
True
INFO:     127.0.0.1:59083 - "GET /predict/?id=2&x=-0.9015375326817245&y=0.11718047485328942 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:59084 - "GET /result/?id=2&result=False HTTP/1.1" 200 OK
3 0.04589805358536947 -0.3668583015147948
True
INFO:     127.0.0.1:59085 - "GET /predict/?id=3&x=0.04589805358536947&y=-0.3668583015147948 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:59086 - "GET /result/?id=3&result=False HTTP/1.1" 200 OK
4 0.449614088484495 0.7687323299276332
True
INFO:     127.0.0.1:59087 - "GET /predict/?id=4&x=0.449614088484495&y=0.7687323299276332 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:59088 - "GET /result/?id=4&result=True HTTP/1.1" 200 OK
5 0.25506980904862986 -0.737378389654872
True
INFO: 

In [11]:
#!pip install uvicorn

  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
